## Mineração de Dados Educacionais – PISA 2018 (Brasil)

Este notebook utiliza três tabelas centrais: `STU_BRA.xlsx`, `FLT_BRA.xlsx` e `SCH_BRA.xlsx` como base para esta etapa de mineração de dados do projeto **Efeito Escola e Gradiente Socioeconômico no Brasil**. Cada arquivo corresponde a um nível analítico distinto e complementar (aluno, desempenho, escola), o que organiza a informação, reduz ruído e favorece a consistência metodológica. A mesma lógica orienta a seleção das variáveis em cada tabela.

Na tabela `STU_BRA.xlsx` (nível aluno), a variável `BELONG` (sentimento de pertencimento) captura o grau em que o estudante se sente parte da comunidade escolar. Algumas fontes pesquisadas sugerem que maior senso de pertencimento se associa a maior engajamento e, em média, a melhores resultados. Nesse contexto, avaliamos `BELONG` ao lado de `ESCS` e `READ`, uma vez que parece plausível que um alto pertencimento atenue o efeito negativo de origens socioeconômicas desfavorecidas ao longo do gradiente. Vai saber. Quer dizer, saberemos!

Os controles de gênero (`ST004D01T`) e de histórico de repetência (`REPEAT`) foram incluídos para tentar separar o efeito do `ESCS` de outras características associadas ao desempenho. Cremos que possa haver, entre meninos e meninas, padrões distintos de proficiência, e como, popularmente, a repetência costuma estar associada a escores mais baixos, ao condicionar-mo-nas ao modelo, pretendemos estimar um efeito condicional do `ESCS`, reduzindo a contaminação por fatores não diretamente socioeconômicos. Reconhecemos, porém, que `REPEAT` pode ser, em parte, consequência do próprio `ESCS`; logo, esse controle tende a subestimar o efeito total do contexto socioeconômico, o que deve ser levado em conta na interpretação.

A variável `ESCS` é um índice sintético que resume a posição socioeconômica, cultural e educacional da família. Ela é construída pela OCDE a partir de modelos de Teoria de Resposta ao Item, combinando informação sobre escolaridade e ocupação dos pais, recursos culturais no domicílio, bens, livros e outros indicadores em uma única medida contínua. Trata-se de um índice do tipo WLE (Estimativa de verossimilhança ponderada), calculado com base no procedimento descrito no [PISA 2018 Technical Report](https://www.oecd.org/pisa/data/pisa2018technicalreport/) (capítulo de Scaling Procedures), aplicado tanto às proficiências quanto aos índices derivados dos questionários.

No banco oficial do PISA, o `ESCS` já é divulgado em uma escala padronizada com média aproximada igual a 0 e desvio-padrão aproximado igual a 1. Para fins de armazenamento nos arquivos, a OCDE aplica uma transformação linear: o valor publicado é dado por valor_no_arquivo = (valor_original + 5) × 1000. Assim, um `ESCS` verdadeiro de `−0,103` aparece como `4897`, enquanto um valor de `+1,2` aparece como `6200`. Por conta disso, nós revertemos esse deslocamento para recuperar a escala oficial aplicando `ESCS = valor_no_arquivo / 1000 - 5`. Só para deixar claro, os fatores 5 e 1000 não são escolhas arbitrárias deste estudo, mas a inversão exata da codificação adotada pela OCDE, também utilizada para outros índices como DISCLIMA, JOYREAD e SCREADCOMP (documentados no [PISA 2018 Database – Codebook e Data Analysis Manual](https://www.oecd.org/pisa/data/2018database/)).

Em termos substantivos, valores mais altos de `ESCS` indicam famílias com maior capital socioeconômico. Neste estudo, tratamo-lo como eixo do gradiente socioeconômico e o coeficiente associado a ele indica como a proficiência em leitura varia em função da origem social do estudante, servindo de referência para quantificar desigualdades educacionais associadas ao contexto socioeconômico.

A variável `READ` representa a proficiência em leitura dos estudantes. Na base `FLT_BRA.xlsx`, o escore é obtido a partir da combinação adequada dos dez valores plausíveis de leitura e dos pesos amostrais correspondentes. Utilizamos `READ` como variável dependente por ser uma medida padronizada (média na OCDE ≈ 500, desvio-padrão ≈ 100), comparável entre alunos e escolas e alinhada ao foco do PISA em compreensão leitora.

Por fim, `STU_BRA.xlsx` fornece os identificadores dos alunos (`CNTSTUID`) e das escolas (`CNTSCHID`), além de `DISCLIMA` (clima disciplinar) e outras variáveis usadas como regressores. Em conjunto, essas informações permitem descrever o gradiente socioeconômico aluno a aluno e conectar esse gradiente às características das escolas, aproximando a análise tanto de evidências empíricas consolidadas quanto das hipóteses teóricas que motivam o estudo.

A tabela `FLT_BRA.xlsx` complementa o nível aluno com as variáveis relacionadas ao desempenho medido pelo PISA e ao desenho amostral. Ela contém `READ` e `READ.SE`, calculados a partir dos dez valores plausíveis de leitura (`PV1READ`…`PV10READ`) gerados pela OCDE, que aplicam a Teoria de Resposta ao Item para lidar com o fato de cada estudante responder apenas parte dos itens do teste. O valor de `READ` sintetiza a proficiência média comparável entre alunos e escolas, enquanto `READ.SE` expressa o erro-padrão associado a essa estimativa, de modo que erros menores indicam maior precisão. A mesma base inclui o peso final do estudante, `SENWT` (*Student Final Weight*), utilizado como fator de expansão: como escolas e alunos são amostrados com probabilidades distintas, aplicar `SENWT` em médias, regressões e modelos garante que os resultados reflitam a população nacional de estudantes de 15 anos, e não apenas a amostra observada.

Além disso, `FLT_BRA.xlsx` reúne índices psicométricos que capturam aspectos intrapessoais relevantes para o desempenho em leitura, como `JOYREAD` e `SCREADCOMP`. Esses índices são derivados do questionário, construídos por escalas WLE e centrados em torno de zero. O índice `JOYREAD` representa o prazer declarado em ler, enquanto `SCREADCOMP` mede a autoeficácia na leitura. Ambos são utilizados para investigar se motivação e autopercepção moderam o gradiente socioeconômico: por exemplo, um aluno de baixo `ESCS`, mas com alta motivação ou confiança, pode apresentar desempenho acima do esperado, enquanto baixa autoeficácia pode amplificar desigualdades.

A planilha `SCH_BRA.xlsx` introduz o nível de contexto escolar, correspondente ao segundo nível dos modelos. Para cada `CNTSCHID`, são disponibilizados, entre outros indicadores, os índices `EDUSHORT` e `STAFFSHORT`, construídos a partir das respostas dos diretores sobre escassez de materiais pedagógicos e de pessoal qualificado. Os valores originais variam de 1 a 119; ao reescalonarmos (`/10 - 5`), recuperamos uma escala aproximadamente centrada em zero que facilita a interpretação, em que valores positivos indicam maior escassez que a média da OCDE e valores negativos indicam menor escassez. A base inclui também `SC016Q01TA` e `SC016Q02TA`, que expressam os percentuais da receita escolar provenientes de fontes governamentais e de contribuições privadas (famílias, doadores, patrocínios), permitindo caracterizar a composição financeira do ambiente escolar.

Esses indicadores são vinculados aos alunos via `CNTSCHID`, o que possibilita testar se condições objetivas de recursos e financiamento moderam a inclinação do gradiente socioeconômico e quantificar quanto da variância entre escolas (efeito escola) está associada a fatores mensuráveis de gestão e infraestrutura. Combinando as três bases, é possível modelar o gradiente socioeconômico ponderando corretamente o desenho amostral e explorando fatores intrapessoais e institucionais que podem suavizar ou acentuar o efeito da origem socioeconômica sobre `READ`.




In [73]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import display
from pathlib import Path
import sys

sns.set_theme(style="whitegrid", palette="colorblind")
pd.set_option("display.max_columns", 30)
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")


Importando nossos scripts de apoio

In [109]:
# Adicionando o diretório scripts ao path do Python

scripts_dir = Path.cwd() / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

import read_excel_sheets as les
import exploratory as expl
import estatisticas as estat

Selecionando as planilhas relevantes

In [75]:
BASE_DIR = Path("pisa2018")
STU_FILE = BASE_DIR / "stu" / "STU_BRA.xlsx"
FLT_FILE = BASE_DIR / "flt" / "FLT_BRA.xlsx"
SCH_FILE = BASE_DIR / "sch" / "SCH_BRA.xlsx"

for path in (STU_FILE, FLT_FILE, SCH_FILE):
    assert path.exists(), f"Arquivo não encontrado: {path}"


Conferindo as abas disponíveis em cada planilha

In [76]:
les.inspect_workbook(STU_FILE)
les.inspect_workbook(FLT_FILE)
les.inspect_workbook(SCH_FILE)

STU_BRA.xlsx -> abas disponíveis: ['data.with.lbl', 'data', 'fields']
FLT_BRA.xlsx -> abas disponíveis: ['data.with.lbl', 'data', 'fields']
SCH_BRA.xlsx -> abas disponíveis: ['data.with.lbl', 'data', 'fields']


## Features Relevantes

Selecionaremos as colunas abaixo, apenas das abas `data` de cada planilha.
- `STU_BRA.xlsx` fornece os indicadores de aluno (`ESCS`, `DISCLIMA`, `BELONG`, gênero, repetência etc.).
- `FLT_BRA.xlsx` agrega os escores plausíveis de leitura (`READ`) e os pesos amostrais (`SENWT`). Os identificadores (`CNTSTUID`) aparecem deslocados em +50.000 e são alinhados manualmente.
- `SCH_BRA.xlsx` traz índices de contexto escolar (`EDUSHORT`, `STAFFSHORT`), além da composição de financiamento (`SC016Qxx`).


Selecionando as colunas relevantes em cada planilha


In [77]:
STU_COLS = [
    "CNTSTUID", "CNTSCHID", "ESCS", "DISCLIMA", "BELONG",
    "ST004D01T", "REPEAT"
]
FLT_COLS = [
    "CNTSTUID", "CNTSCHID", "SENWT", "READ", "READ.SE",
    "JOYREAD", "SCREADCOMP"
]
SCH_COLS = [
    "CNTSCHID", "SC016Q01TA", "SC016Q02TA", "EDUSHORT", "STAFFSHORT"
]

Criando os dataframes

In [78]:
stu = les.load_sheet(STU_FILE, STU_COLS)
flt = les.load_sheet(FLT_FILE, FLT_COLS)
sch = les.load_sheet(SCH_FILE, SCH_COLS)

stu.shape, flt.shape, sch.shape

((10691, 7), (8311, 7), (597, 5))

De acordo com a documentação já mencionada, o atributo `CNTSTUID` na planilha `FLT` vem com um acréscimo de 50.000. Em vez de 7600001, 7600002, etc., os IDs aparecem como 7650001, 7650002, etc. Por conta disso, para regularizar a situação, precisamos subtrair 50.000 nesses IDs. Caso contrário,  o merge aluno‑a‑aluno não funcionará corretamente. Na verdade, o merge não funcionará de jeito nenhum, porque não haverá correspondência entre os IDs. Vejamos isso usando `how="outer"` para reter todos os registros que estão só em stu, só em flt e os que aparecem nos dois. Isso nos permitirá medir a qualidade do merge a partir dos percentuais de perda/ganho, por exemplo. 

In [79]:
print("AVALIAÇÃO DO MERGE SEM AJUSTE\n")
#print(f"Dimensões originais:")
#print(f"   • STU_BRA: {stu.shape[0]:,} alunos × {stu.shape[1]} variáveis")
#print(f"   • FLT_BRA: {flt.shape[0]:,} alunos × {flt.shape[1]} variáveis\n")

# Identificar o problema do deslocamento
print("Compatibilidade dos IDs:")
print(f"   STU - Primeiros CNTSTUID: {stu['CNTSTUID'].head(3).tolist()}")
print(f"   FLT - Primeiros CNTSTUID: {flt['CNTSTUID'].head(3).tolist()}")
print(f"\nDiferença: {flt['CNTSTUID'].iloc[0] - stu['CNTSTUID'].iloc[0]:,}")

merge_diagnostic_raw = stu.merge(
    flt, 
    on=["CNTSTUID", "CNTSCHID"], 
    how="outer", 
    indicator=True,
    suffixes=('_stu', '_flt')
)

merge_counts_raw = merge_diagnostic_raw['_merge'].value_counts()
print(f"\nAlunos:")
for status, count in merge_counts_raw.items():
    pct = 100 * count / len(merge_diagnostic_raw)
    status_label = {
        'both': 'Combinados (STU x FLT):',
        'left_only': 'Apenas em STU',
        'right_only': 'Apenas em FLT'
    }.get(status, status)
    print(f"   {status_label}: {count:,} ({pct:.1f}%)")

#if merge_counts_raw.get('both', 0) == 0:
#    print("\nNenhuma combinação válida encontrada!")

# Limpar diagnóstico preliminar
del merge_diagnostic_raw

AVALIAÇÃO DO MERGE SEM AJUSTE

Compatibilidade dos IDs:
   STU - Primeiros CNTSTUID: [7600001, 7600002, 7600005]
   FLT - Primeiros CNTSTUID: [7650001, 7650002, 7650003]

Diferença: 50,000

Alunos:
   Apenas em STU: 10,691 (56.3%)
   Apenas em FLT: 8,311 (43.7%)
   Combinados (STU x FLT):: 0 (0.0%)


Ajustando o deslocamento: subtraindo 50000 do CNTSTUID em `FLT`

In [80]:
flt["CNTSTUID"] = flt["CNTSTUID"] - 50_000
print(f"Primeiros IDs após correção: {flt['CNTSTUID'].head(3).tolist()}\n")

Primeiros IDs após correção: [7600001, 7600002, 7600003]



Vamos repetir o merge, com `how="outer"`, e conferir o resultado.

In [81]:
print("AVALIAÇÃO DO MERGE PÓS-AJUSTE\n")

print("Compatibilidade dos IDs:")
print(f"   STU - Primeiros CNTSTUID: {stu['CNTSTUID'].head(3).tolist()}")
print(f"   FLT - Primeiros CNTSTUID: {flt['CNTSTUID'].head(3).tolist()}")
diferenca = flt['CNTSTUID'].iloc[0] - stu['CNTSTUID'].iloc[0]
print(f"\nDiferença: {diferenca:,}")

#if diferenca == 0:
#    print("IDs alinhados!\n")
#else:
#    print(f"Ainda há desalinhamento de {abs(diferenca):,}\n")

merge_diagnostic = stu.merge(
    flt, 
    on=["CNTSTUID", "CNTSCHID"], 
    how="outer", 
    indicator=True,
    suffixes=('_stu', '_flt')
)

merge_counts = merge_diagnostic['_merge'].value_counts()
print("\nAlunos:")
for status, count in merge_counts.items():
    pct = 100 * count / len(merge_diagnostic)
    status_label = {
        'both': 'Combinados (STU x FLT):',
        'left_only': 'Apenas em STU',
        'right_only': 'Apenas em FLT'
    }.get(status, status)
    print(f"   {status_label}: {count:,} ({pct:.1f}%)")

# Limpar diagnóstico preliminar
del merge_diagnostic

AVALIAÇÃO DO MERGE PÓS-AJUSTE

Compatibilidade dos IDs:
   STU - Primeiros CNTSTUID: [7600001, 7600002, 7600005]
   FLT - Primeiros CNTSTUID: [7600001, 7600002, 7600003]

Diferença: 0

Alunos:
   Apenas em STU: 5,783 (41.0%)
   Combinados (STU x FLT):: 4,908 (34.8%)
   Apenas em FLT: 3,403 (24.1%)


## Análise do merge

Se olharmos para o resultado do merge com how="inner", que retorna apenas as linhas onde ambas as chaves (CNTSTUID E CNTSCHID) coincidem nos dois DataFrames: 4.908 alunos, apenas, parece que há uma diferença considerável entre as bases STU e FLT. Ao menos entre os IDs dos alunos. Ainda que em um primeiro momento isso tenha ligado um sinal de alerta em relação à manutenção do JOIN, avaliamos que, apesar da base `FLT` derivar da `STU`, elas não são idênticas. `FLT` contém apenas os alunos que participaram do teste de leitura, enquanto que `STU` inclui todos os alunos amostrados, independentemente de terem participado ou não do teste de leitura. Portanto, pareceu-nos adequado e até mesmo esperado que FLT seja uma subamostra da base STU. Ou seja, dá para manter o JOIN, mesmo com a redução do número de alunos, já que isso nos garante integridade dos dados de proficiência em leitura. Do contrário, ainda que o aluno tenha `ESCS` em `STU`, se ele não tiver `READ` em `FLT`, não poderemos usá-lo na análise, uma vez que o gradiente socioeconômico depende de ambos os dados. Por tudo isso, seguimos com o merge definitivo.

Só para não dizer que não exlicamos o óbvio, e pecar por excesso, dizem, é menos "ruim" do que pecar por sua falta, vamos combinar as colunas dos dataframes`stu` (características do aluno) com as colunas do dataframe `flt` (escore de leitura, peso amostral, índices motivacionais) em um  único dataframe: `students`. A chave do merge são as colunas `CNTSTUID` e `CNTSCHID`, que acabamos de ajeitar, de tal modo que cada aluno só entra se tiver uma linha correspondente nas duas bases. Para isso usaremos o  `validate="one_to_one"`, que impõe uma checagem importante, forçando o Pandas a verificar se cada combinação de aluno-escola aparece no máximo uma vez em cada DataFrame. **Desse modo, caso haja duplicados, o merge falha, impedindo que propaguemos dados replicados sem perceber.** 

In [82]:
students = (
    stu.merge(flt, on=["CNTSTUID", "CNTSCHID"], how="inner", validate="one_to_one")
)

students.shape
students.head()


,CNTSTUID,CNTSCHID,ST004D01T,REPEAT,ESCS,DISCLIMA,BELONG,JOYREAD,SCREADCOMP,SENWT,READ,READ.SE
0,7600001,7600614,Male,Did not repeat a grade,"3,353.000",532.000,NaN,623.000,NaN,0.308,470.375,22.412
1,7600002,7600190,Female,Did not repeat a grade,"7,479.000",870.000,"1,014.000",194.000,16.000,0.342,432.607,23.037
2,7600010,7600048,Female,Did not repeat a grade,"6,273.000",47.000,"1,534.000",236.000,54.000,0.525,428.324,31.055
3,7600020,7600444,Male,Repeated a grade,"1,830.000",313.000,"1,185.000",393.000,35.000,0.658,378.231,24.029
4,7600022,7600047,Male,Repeated a grade,"1,533.000",63.000,392.000,14.000,48.000,0.511,419.672,16.979


Logo em seguida precisaremos restaurar a escala original de alguns índices, mas antes disso vamos inspecionar os dados únicos nas variáveis categóricas.

In [83]:
expl.resumo_categoricas(students)

Variáveis categóricas: ST004D01T, REPEAT 

[ST004D01T]  (níveis: 2, missing: 0)
  Female                          2533  ( 51.6%)
  Male                            2375  ( 48.4%)

[REPEAT]  (níveis: 2, missing: 111)
  Did not repeat a  grade         3316  ( 67.6%)
  Repeated a  grade               1481  ( 30.2%)
  <NaN>                            111  (  2.3%)



### Reescalonamento dos Índices WLE

Como mencionado, precisamos **reescalar os dados** para recuperar os valores originais daqueles índices WLE selecionados, tal como definidos pela OCDE. Vejamos a lógica para interpretação de cada variável.

#### `ESCS`

Exemplo de reescalonamento:

* Valor original no arquivo: `4897`
* Divisão por 1000: `4897 / 1000 = 4.897`
* Subtração de 5: `4.897 - 5 = -0.103`

Interpretação:

* `-0.103` → família levemente abaixo da média da OCDE
* `+1.2` → família bem acima da média (alto status)
* `-2.0` → família em situação muito vulnerável


#### `DISCLIMA`

Exemplo de reescalonamento:

* Valor original: `478`
* Divisão por 100: `478 / 100 = 4.78`
* Subtração de 5: `4.78 - 5 = -0.22`

Interpretação:

* Valores negativos → clima mais ordenado que a média
* Valores positivos → clima mais problemático
* Exemplo: `-0.22` indica disciplina ligeiramente melhor que a média

#### `JOYREAD`

Exemplo de reescalonamento:

* Valor original: `530`
* Divisão por 100: `530 / 100 = 5.30`
* Subtração de 5: `5.30 - 5 = +0.30`

Interpretação:

* `+0.30` → gosta mais de ler que a média da OCDE
* `-0.50` → gosta menos de ler
* Valores típicos: aproximadamente de `-3` a `+3`

#### `SCREADCOMP`

Exemplo de reescalonamento:

* Valor original: `462`
* Divisão por 100: `462 / 100 = 4.62`
* Subtração de 5: `4.62 - 5 = -0.38`

Interpretação:

* `-0.38` → confiança abaixo da média na capacidade de ler
* `+1.0` → alta autoeficácia
* Pode moderar o efeito do `ESCS` (um aluno em contexto pobre, mas confiante, pode apresentar melhor desempenho do que o esperado pelo gradiente socioeconômico)

#### `BELONG`

Exemplo de reescalonamento:

* Valor original: `980`
* Divisão por 100: `980 / 100 = 9.80`
* **Sem** subtração de 5

Interpretação:

* `9.80` → forte senso de pertencimento à escola
* `5.00` → pertencimento moderado
* `2.00` → sente-se excluído ou isolado
* Hipótese: alto pertencimento pode amortecer o efeito negativo de baixo `ESCS`

### Variáveis Categóricas Codificadas (`dummies`)

#### `GENDER` → `gender_male`

* Valor original: `"Male"` ou `"Female"`
* Comparação lógica: `"Male" == 'Male'` → `True`; `"Female" == 'Male'` → `False`
* Conversão para inteiro: `True → 1`, `False → 0`

Valores finais:

* `1` = Menino (masculino)
* `0` = Menina (feminino)


#### `REPEAT` → `repeat_flag`

* Valor original:

  * `"Repeated a  grade"`
  * `"Did not repeat a  grade"`
  * `NaN` (sem informação)

* Recodificação lógica:

  * `repeat_flag = 1` se `REPEAT == "Repeated a  grade"`
  * `repeat_flag = 0` se `REPEAT == "Did not repeat a  grade"`
  * `repeat_flag = 2` se `REPEAT` estiver ausente

Valores finais:

* `1` = Repetiu pelo menos uma série
* `0` = Nunca repetiu
* `2` = Informação ausente (casos mantidos como *missing* e tratados como categoria própria)


### 📈 Resumo Visual dos Valores Esperados

| Variável         | Transformação            | Escala final               | Exemplo             |
| ---------------- | ------------------------ | -------------------------- | ------------------- |
| `escs_std`       | `/1000 - 5`              | ≈ -3 a +3 (média = 0)      | `-0.103`            |
| `disclima_std`   | `/100 - 5`               | ≈ -5 a +5 (média = 0)      | `-0.22`             |
| `joyread_std`    | `/100 - 5`               | ≈ -3 a +3 (média = 0)      | `+0.30`             |
| `screadcomp_std` | `/100 - 5`               | ≈ -3 a +3 (média = 0)      | `-0.38`             |
| `belong_index`   | `/100`                   | 0 a 10 (não centrada em 0) | `9.80`              |
| `gender_male`    | `== "Male"`              | 0 ou 1                     | `1` (menino)        |
| `repeat_flag`    | `== "Repeated a  grade"` | 0, 1 ou `2`                | `0` (nunca repetiu) |




Vamos fazer uma cópia por segurança, mantendo o nome do dataframe como `students` mesmo, que fica visualmente mais agradável.

In [84]:
students_raw = students.copy()  # backup do dado bruto

In [85]:
students = students_raw.copy()  # dataset de trabalho
students.head()

,CNTSTUID,CNTSCHID,ST004D01T,REPEAT,ESCS,DISCLIMA,BELONG,JOYREAD,SCREADCOMP,SENWT,READ,READ.SE
0,7600001,7600614,Male,Did not repeat a grade,"3,353.000",532.000,NaN,623.000,NaN,0.308,470.375,22.412
1,7600002,7600190,Female,Did not repeat a grade,"7,479.000",870.000,"1,014.000",194.000,16.000,0.342,432.607,23.037
2,7600010,7600048,Female,Did not repeat a grade,"6,273.000",47.000,"1,534.000",236.000,54.000,0.525,428.324,31.055
3,7600020,7600444,Male,Repeated a grade,"1,830.000",313.000,"1,185.000",393.000,35.000,0.658,378.231,24.029
4,7600022,7600047,Male,Repeated a grade,"1,533.000",63.000,392.000,14.000,48.000,0.511,419.672,16.979


Agora vamos escalar

In [86]:
students["escs_std"] = students["ESCS"] / 1000 - 5  # índice socioeconômico centrado em 0
students["disclima_std"] = students["DISCLIMA"] / 100 - 5  # clima disciplinar (≈-5 a +5)
students["joyread_std"] = students["JOYREAD"] / 100 - 5
students["screadcomp_std"] = students["SCREADCOMP"] / 100 - 5
students["belong_index"] = students["BELONG"] / 100  # escala 0-10 (bem-estar)

# Gênero: 1 = menino, 0 = menina
students["gender_male"] = (students["ST004D01T"] == "Male").astype("int8")

# Repetência: 0 = não repetiu, 1 = repetiu, 2 = sem informação
students["repeat_flag"] = students["REPEAT"].map({
    "Did not repeat a  grade": 0,
    "Repeated a  grade": 1
})
students["repeat_flag"] = students["repeat_flag"].fillna(2).astype("int8")

In [87]:
students.head()

,CNTSTUID,CNTSCHID,ST004D01T,REPEAT,ESCS,DISCLIMA,BELONG,JOYREAD,SCREADCOMP,SENWT,READ,READ.SE,escs_std,disclima_std,joyread_std,screadcomp_std,belong_index,gender_male,repeat_flag
0,7600001,7600614,Male,Did not repeat a grade,"3,353.000",532.000,NaN,623.000,NaN,0.308,470.375,22.412,-1.647,0.320,1.230,NaN,NaN,1,0
1,7600002,7600190,Female,Did not repeat a grade,"7,479.000",870.000,"1,014.000",194.000,16.000,0.342,432.607,23.037,2.479,3.700,-3.060,-4.840,10.140,0,0
2,7600010,7600048,Female,Did not repeat a grade,"6,273.000",47.000,"1,534.000",236.000,54.000,0.525,428.324,31.055,1.273,-4.530,-2.640,-4.460,15.340,0,0
3,7600020,7600444,Male,Repeated a grade,"1,830.000",313.000,"1,185.000",393.000,35.000,0.658,378.231,24.029,-3.170,-1.870,-1.070,-4.650,11.850,1,1
4,7600022,7600047,Male,Repeated a grade,"1,533.000",63.000,392.000,14.000,48.000,0.511,419.672,16.979,-3.467,-4.370,-4.860,-4.520,3.920,1,1


Vamos remover as colunas desnecessárias (originais) e renomear as novas para manter a consistêcias

In [88]:
# Remover colunas originais brutas que já têm versão transformada
cols_drop = [
    "ESCS", "DISCLIMA", "JOYREAD", "SCREADCOMP", "BELONG",
    "ST004D01T", "REPEAT"
]

students = students.drop(columns=cols_drop)

# Renomeaando as colunas novas para manter o mesmo esquema de nomes

students = students.rename(columns={
    "escs_std": "ESCS",              # índice socioeconômico padronizado
    "disclima_std": "DISCLIMA",      # clima disciplinar padronizado
    "joyread_std": "JOYREAD",        # prazer pela leitura padronizado
    "screadcomp_std": "SCREADCOMP",  # autoeficácia em leitura padronizada
    "belong_index": "BELONG",        # pertencimento (0–10)
    "gender_male": "ST004D01T",      # 1 = menino, 0 = menina
    "repeat_flag": "REPEAT"          # 0 = não repetiu, 1 = repetiu, 2 = sem info
})

# Reorganizando as colunas na ordem do schema original

cols_order = [
    "CNTSTUID", "CNTSCHID",
    "ST004D01T", "REPEAT",
    "ESCS", "DISCLIMA", "BELONG", "JOYREAD", "SCREADCOMP",
    "SENWT", "READ", "READ.SE"
]

students = students[cols_order]

students.head()



,CNTSTUID,CNTSCHID,ST004D01T,REPEAT,ESCS,DISCLIMA,BELONG,JOYREAD,SCREADCOMP,SENWT,READ,READ.SE
0,7600001,7600614,1,0,-1.647,0.320,NaN,1.230,NaN,0.308,470.375,22.412
1,7600002,7600190,0,0,2.479,3.700,10.140,-3.060,-4.840,0.342,432.607,23.037
2,7600010,7600048,0,0,1.273,-4.530,15.340,-2.640,-4.460,0.525,428.324,31.055
3,7600020,7600444,1,1,-3.170,-1.870,11.850,-1.070,-4.650,0.658,378.231,24.029
4,7600022,7600047,1,1,-3.467,-4.370,3.920,-4.860,-4.520,0.511,419.672,16.979


Como você deve ter percebido, preferimos manter os dados ausentes em `REPEAT`, tratando-os como categoria (2) por que adiante podemos incluí-los tanto em uma quanto em outra, de tal modo que podemos avaliar a repetência somando os (não informados) aos qque repetiram, bem como aos que não repetiram. Cremos que, com isso, podemos capturar nuances que seriam perdidas caso fossem removidos. Contudo, a mesma lógica não se aplica as demais variáveis (veja abaixo), onde os dados ausentes devem ser tratados via de regra, pela média, mediana ou por uma imputação múltipla, em vez de serem excluídos das análises.

In [89]:
students.isnull().sum()

CNTSTUID         0
CNTSCHID         0
ST004D01T        0
REPEAT           0
ESCS           106
DISCLIMA       266
BELONG        1114
JOYREAD        478
SCREADCOMP     732
SENWT            0
READ             0
READ.SE          0
dtype: int64

Entretanto, por mais que essa decisão pareça simples, principalmente para o caso de `ESCS`, cujo número de ausentes é percentualmente baixo, para formalizar nossa escolha avaliamos as três estratégias clássicas mencionadas: **exclusão de casos**, **imputação simples (média/mediana)** e **imputação múltipla**. Cada uma delas parte de pressupostos distintos sobre o mecanismo que gera os dados ausentes, o que impacta, de forma direta e desigual, a validade dos resultados.

Para encurtar a conversa, por que simplesmente não criamos uma categoria “faltante”, como fizemos em `REPEAT`? Porque, naquele caso, transformar o ausente em uma categoria explícita (2) é plausível: `REPEAT` é uma variável categórica, e o grupo “não informado” pode ser tratado como categoria substantivamente interpretável ou usado em análises de sensibilidade (agregando-o a quem repetiu, a quem não repetiu, ou mantendo-o separado).

Essa lógica, porém, **não se aplica** às demais variáveis, cujos indicadores foram construídos como **índices contínuos**, muitas vezes com propriedades métricas definidas (por exemplo, sob modelos de Teoria de Resposta ao Item). Criar neles uma categoria “faltante” equivaleria a inventar um nível artificial que não existe na escala original. Intuitivamente, isso desloca a régua sem mudar o fenômeno. Tecnicamente, tende a distorcer a métrica, quebrar interpretações e introduzir descontinuidades espúrias.

Seria como, em uma escala de proficiência de leitura, adicionar um ponto “666 = não respondeu” e exigir que o modelo trate esse valor como se fosse um nível real de desempenho. Computacionalmente funciona; conceitualmente é torto. A literatura metodológica converge em desaconselhar esse tipo de codificação: em variáveis contínuas, é mais adequado imputar valores ausentes a partir da informação observada ou modelar explicitamente o mecanismo de ausência, em vez de criar categorias fictícias.

Mas quando a exclusão é aceitável então, já que falamos que no caso de `ESCS` poderíamos fazê-lo? Aí que o bicho pega. A decisão entre excluir ou imputar dados ausentes depende do **mecanismo de missing** subjacente, que pode ser classificado em três tipos principais:  MCAR, MAR e MNAR.

De forma resumida:

* **MCAR (Missing Completely At Random)**:
  A probabilidade de um dado estar ausente **não depende** nem de valores observados, nem dos não observados. O missing é puro azar.
  → Aqui, a exclusão de casos é estatisticamente “limpa”: não gera viés, apenas reduz o tamanho da amostra.

* **MAR (Missing At Random)**:
  A probabilidade de ausência **depende de outras variáveis observadas**, mas não do valor verdadeiro faltante, depois de controladas essas variáveis.
  Ex.: alunos com determinado perfil socioeconômico ou em certas escolas têm maior chance de não responder `JOYREAD`.
  → Exclusão simples já pode induzir viés; métodos como imputação múltipla tornam-se preferíveis.

* **MNAR (Missing Not At Random)**:
  A probabilidade de ausência **depende do próprio valor ausente** (ou de algo não observado).
  Ex.: alunos com baixíssima motivação para leitura evitam responder itens sobre leitura.
  → Nesse caso, tanto exclusão quanto imputações ingênuas podem ser enviesadas; idealmente, precisa-se de modelos mais sofisticados ou análises de sensibilidade.

Na prática aplicada em educação, parece que assumir `MCAR` é quase sempre otimista demais, mas não é só isso e, ainda que não exista um “bala de prata”, algumas **regras de bolso razoáveis** ajudam, como por exemplo:

* Até **≈5% de ausentes**, sob algo próximo de MCAR, a **exclusão de casos é geralmente aceitável**.
* Entre **5% e 10%**, exige mais cautela: exclusão pode ser usada em análises simples, mas já vale considerar imputação, sobretudo se a variável for central.
* Acima de **10%**, especialmente quando há indícios de MAR/MNAR, a exclusão sistemática tende a produzir vieses relevantes e perda substantiva de poder estatístico; aqui, **imputação múltipla** passa a ser a estratégia recomendada.

Aplicando isso ao nosso conjunto:

* `ESCS` ≈ **2.2%** faltantes →
  Excluir esses casos é aceitável: a perda é pequena, e, de todo modo, sem `ESCS` o aluno não pode contribuir para o gradiente socioeconômico. Nesse contexto, a exclusão não distorce o estimando, apenas reduz ligeiramente o n útil.

* `DISCLIMA` ≈ **5.4%** →
  Estamos na fronteira. Exclusão pura é possível, mas já começa a ser discutível, principalmente porque `DISCLIMA` é uma variável-chave.

* `JOYREAD` ≈ **9.7%**, 
* `SCREADCOMP` ≈ **14.9%**, 
* `BELONG` ≈ **22.7%** →
  Aqui, a exclusão massiva seria metodologicamente cara, já que reduziria o tamanho da amostra, concentraria a análise em perfis mais completos (e possivelmente mais favorecidos) e aumentaria o risco de viés de seleção. Nesses casos, manter “quem não respondeu” como se não existisse é, na melhor das hipóteses, ingênuo. Logo, tudo isso justifica nossa opção final: **imputação múltipla** para esses índices psicossociais.

Mas na prática, o que significa isso? Significa que **respeitaremos a estrutura dos dados** utilizando informações de aluno, escola, desempenho (`READ`), contexto (`ESCS`), clima (`DISCLIMA`), atitudes (`JOYREAD`, `SCREADCOMP`, `BELONG`), repetência e variáveis de nível escola para gerar valores plausíveis para os ausentes; **preservaremos variabilidade e relações**, já que ao invés de substituir tudo por uma média única, geraremos várias versões imputadas do banco, incorporando incertezas, de tal modo que as estimativas finais reflitam tanto a variabilidade quanto a incerteza sobre os valores imputados e, finalmente, continuaremos alinhados com a prática metodológica consolidada em avaliação educacional, haja vista o compromisso entre rigor estatístico e o respeito às nuances dos dados, à redução de viés sem sacrificar a estrutura hierárquica nem a interpretabilidade dos resultados.

In [90]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

# Variáveis com ausentes que serão imputadas
vars_impute = ["ESCS", "DISCLIMA", "JOYREAD", "SCREADCOMP", "BELONG"]

# Variáveis preditoras: não serão imputadas nem alteradas conceitualmente
vars_aux = [
    "READ", "SENWT",
    "REPEAT",        # já codificada (0,1,2) ou 0/1, completa
    "ST004D01T",     # já codificada (0/1), completa
]

# Matriz para o IterativeImputer:
# As colunas em vars_impute têm NA
# As colunas em vars_aux entram como preditores (sem NA)

X = students[vars_impute + vars_aux]

imp = IterativeImputer(
    max_iter=20,
    sample_posterior=True,   # essencial para múltiplas imputações
    random_state=123
)

m = 20
imputations = []

for s in range(m):
    # nova semente a cada imputação
    imp.random_state = 123 + s
    
    # ajusta o modelo e imputa SOMENTE onde há NaN
    X_imp = imp.fit_transform(X)
    
    # cria cópia do dataframe original
    students_imp = students.copy()

    # sobrescreve apenas as variáveis imputadas com os valores imputados (nas posições correspondentes)
    students_imp[vars_impute] = X_imp[:, :len(vars_impute)]
    
    # garante que REPEAT e ST004D01T continuem categóricos como definidos
    students_imp["REPEAT"] = students_imp["REPEAT"].astype(students["REPEAT"].dtype)
    students_imp["ST004D01T"] = students_imp["ST004D01T"].astype(students["ST004D01T"].dtype)
    
    imputations.append(students_imp)


Como de constume, vamos conferir a parada:

In [91]:
students_imp.isnull().sum()

CNTSTUID      0
CNTSCHID      0
ST004D01T     0
REPEAT        0
ESCS          0
DISCLIMA      0
BELONG        0
JOYREAD       0
SCREADCOMP    0
SENWT         0
READ          0
READ.SE       0
dtype: int64

## As variáveis de nível 2: Escola

Até agora trabalhamos apenas com variáveis de **nível aluno** (`ESCS`, gênero, repetência, clima disciplinar etc.). Isso é suficiente para modelos “clássicos” de regressão, mas insuficiente quando queremos levar a sério o fato óbvio (embora frequentemente ignorado nas análises) de que **alunos estão agrupados em escolas**. Para capturar esse contexto, precisamos trazer para o dataset dos alunos informações da escola onde eles estudam. Por conta disso, incorporamos ao modelo variáveis de **nível escola**, extraídas da planilha `SCH_BRA.xlsx`, de modo a respeitar a estrutura hierárquica dos dados.

A base `SCH_BRA.xlsx` fornece, para cada `CNTSCHID`, índices de contexto escolar, entre os quais destacamos:

- `EDUSHORT` e `STAFFSHORT`: índices construídos a partir das respostas dos diretores sobre escassez de materiais pedagógicos e de pessoal qualificado. Nos microdados, esses índices são armazenados em uma escala deslocada, semelhante ao que vimos em `FLT`. Aplicamos, portanto, a transformação inversa (`/10 - 5`) para obter `edushort_std` e `staffshort_std`, aproximadamente centrados em 0. Valores positivos indicam maior escassez que a média da OCDE; valores negativos indicam menor escassez (condições mais favoráveis).

- `SC016Q01TA` e `SC016Q02TA`: percentuais da receita escolar provenientes, respectivamente, de fontes governamentais e de contribuições privadas (famílias, doadores, patrocínios), o que nos permite caracterizar a composição do financiamento escolar.

Por construção, essas variáveis compõem o **nível 2** dos modelos, enquanto `ESCS`, gênero, repetência, clima disciplinar, atitudes e proficiência permanecem no **nível 1** (aluno). A junção entre bases nos permite separar o que é efeito das **características individuais dos estudantes** do que é efeito das **condições das escolas**, além de testar se o contexto escolar amplifica, atenua ou modifica o gradiente socioeconômico associado ao `ESCS`. Sem essas variáveis de nível escola, qualquer inferência sobre “impacto da escola” tenderia a confundir diferenças de composição (quem estuda onde) com diferenças reais de oportunidade educacional.


Ante, porém, vamos conferir os dados ausentes e reescalonar os índices WLE conforme necessário.

In [92]:
sch.isnull().sum()

CNTSCHID        0
SC016Q01TA     59
SC016Q02TA    172
EDUSHORT       42
STAFFSHORT     45
dtype: int64

Antes de incorporar as variáveis de nível escola ao dataset de alunos, avaliamos o padrão de dados ausentes em `SCH_BRA.xlsx`. Entre as 597 escolas, observamos:

- `SC016Q01TA`: 59 ausentes → ≈ 9,9%
- `SC016Q02TA`: 172 ausentes → ≈ 28,8%
- `EDUSHORT`: 42 ausentes → ≈ 7,0%
- `STAFFSHORT`: 45 ausentes → ≈ 7,5%

Retomando nossa régua de decisão: até ≈5% de ausentes, a exclusão de casos tende a ser aceitável; entre 5% e 10%, a exclusão já exige cautela e a imputação passa a ser uma alternativa razoável; acima de 10%, a exclusão sistemática costuma induzir vieses relevantes e perda desnecessária de informação.

Aplicando esse critério ao contexto escolar:

- Para `EDUSHORT` e `STAFFSHORT` (7–8% de ausentes), a exclusão não seria absurda, mas, por se tratarem de variáveis centrais para caracterizar recursos e condições de oferta, preferimos não descartar escolas. A opção metodologicamente mais consistente também parece ser tratá-las com **imputação múltipla**, preservando o desenho hierárquico.
  
- Para `SC016Q01TA` (~10% de ausentes), estamos na borda superior da zona de conforto. A imputação múltipla é defensável, sobretudo porque a variável tem interpretação clara (financiamento governamental) e potencial explicativo relevante no nível escola.
  
- Já `SC016Q02TA` (~28,8% de ausentes) acende um alerta mais forte. Quase um terço das escolas não informa a parcela de financiamento privado, o que implica:
  - forte dependência das imputações em relação ao modelo especificado e às suposições de *MAR*;
  - alta fração de informação perdida associada a essa variável, produzindo estimativas mais instáveis (erros-padrão maiores, intervalos de confiança mais largos, maior sensibilidade a pequenas mudanças no modelo).
  
Em outras palavras, `SC016Q02TA` pode ser imputada, mas qualquer conclusão substantiva apoiada nela carrega incerteza adicional. Por isso, optamos por tratá-la principalmente como **variável de análise de sensibilidade**, e não como pilar do modelo principal: usamos sua inclusão ou exclusão (com imputação) para verificar se os resultados centrais se mantêm robustos, em vez de basear inferências fortes em uma covariável com alta proporção de valores imputados.

Um ponto técnico importante é que essa decisão é tomada **antes** do `merge` com o nível aluno. Caso contrário, cada escola com dados ausentes em `EDUSHORT`, `STAFFSHORT` ou nos indicadores de financiamento propagaria *NaN* para todos os seus alunos, inflando artificialmente a taxa de ausência no nível 1. Tratá-los, portanto, se mostra essencial para preservar a integridade da estrutura hierárquica e reduzir vieses na etapa de modelagem.

Só  para registrar: “adotamos abaixo imputação múltipla com m = 20, de modo a representar adequadamente a incerteza associada aos dados ausentes, em vez de tratar a imputação como se gerasse valores ‘certos’.” 

Mas antes disso, vamos fazer uma cópia de segurança.

In [93]:
sch_raw = sch.copy()

In [94]:
sch = sch_raw.copy() 

Imputando, reescalando e renomeando as variáveis de nível escola

In [95]:
# variáveis a imputar em nível escola
vars_impute = ["SC016Q01TA", "SC016Q02TA", "EDUSHORT", "STAFFSHORT"]

X_sch = sch[vars_impute]

imp_sch = IterativeImputer(
    max_iter=20,
    sample_posterior=True,
    random_state=123
)

m = 20  # número de bancos imputados
sch_imputations = []

for k in range(m):
    imp_sch.random_state = 123 + k
    
    # Imputando SC016Q01TA, SC016Q02TA, EDUSHORT, STAFFSHORT
    X_sch_imp = imp_sch.fit_transform(X_sch)
    
    sch_imp_k = sch.copy()
    sch_imp_k[vars_impute] = X_sch_imp

    # Reescalando EDUSHORT e STAFFSHORT (agora já imputados)
    sch_imp_k["EDUSHORT_std"]   = sch_imp_k["EDUSHORT"] / 10 - 5
    sch_imp_k["STAFFSHORT_std"] = sch_imp_k["STAFFSHORT"] / 10 - 5

    sch_imputations.append(sch_imp_k)

sch_imp = sch_imputations[0].copy()

# Substituir as colunas originais pelas versões padronizadas para manter o mesmo esquema de nomes

sch_imp = (
    sch_imp
    .drop(columns=["EDUSHORT", "STAFFSHORT"])  # remove escala deslocada
    .rename(columns={
        "EDUSHORT_std": "EDUSHORT",
        "STAFFSHORT_std": "STAFFSHORT"
    })[["CNTSCHID", "SC016Q01TA", "SC016Q02TA", "EDUSHORT", "STAFFSHORT"]]
)

Conferindo se funcionou.

In [96]:
sch_imp.isnull().sum()

CNTSCHID      0
SC016Q01TA    0
SC016Q02TA    0
EDUSHORT      0
STAFFSHORT    0
dtype: int64

In [97]:
sch_imp.head()

,CNTSCHID,SC016Q01TA,SC016Q02TA,EDUSHORT,STAFFSHORT
0,7600001,20.000,1.000,4.300,4.800
1,7600002,20.000,1.000,6.400,1.800
2,7600003,2.688,19.000,-4.900,-4.800
3,7600005,1.000,19.000,-4.900,-3.400
4,7600006,17.000,1.000,-4.800,-4.800


Agora que já reescalonamos e renomeamos as variáveis de nível escola, podemos incorporá-las ao dataset de alunos via `CNTSCHID`. Assim como antes, usaremos `validate="many_to_one"` para garantir que cada escola seja vinculada corretamente aos seus alunos, sem duplicações inesperadas. Contudo, vamos examinar se há alguma discrepância entre as escolas listadas em `students` e aquelas presentes em `schools`, fazendo um merge preliminar com `how="outer"`.

Observação. No processo de imputação geramos um dataframe chamado `sch_imp` e é esse que usaremos para o merge. Entretanto, para fins de informação, usaremos o nome do dataframe original `sch`, apenas para facilitar o entendimento do resultado.

In [98]:
print("AVALIAÇÃO DO MERGE STUDENTS x SCH\n")

print("Dimensões atuais:")
print(f"   • students : {students_imp.shape[0]:,} linhas × {students_imp.shape[1]} colunas")
print(f"   • sch  : {sch_imp.shape[0]:,} linhas × {sch_imp.shape[1]} colunas\n")

# CNTSCHID apenas em sch
dups_sch = sch_imp["CNTSCHID"].duplicated().sum()
if dups_sch == 0:
    print("Não há escolas repetidas.")
else:
    print(f"ATENÇÃO! Há {dups_sch} escolas repetidas.")
print()

merge_diag = students_imp.merge(
    sch_imp,
    on="CNTSCHID",
    how="outer",
    indicator=True,
    suffixes=("_stu", "_sch")
)

# Ordenando os status para impressão consistente
status_labels = {
    "both": "Escolas combinadas:",
    "left_only": "Apenas em students (aluno sem escola em sch)",
    "right_only": "Apenas em sch (escola sem aluno em students)"
}

print("Resumo do merge:")
total = len(merge_diag)
for key in ["both", "left_only", "right_only"]:
    if key in merge_diag["_merge"].values:
        count = (merge_diag["_merge"] == key).sum()
        pct = 100 * count / total
        print(f"   • {status_labels[key]}: {count:,} ({pct:.2f}%)")
print()

# nível escola
students_sch_ids = set(students["CNTSCHID"].unique())
sch_imp_ids      = set(sch_imp["CNTSCHID"].unique())

only_in_students = students_sch_ids - sch_imp_ids
only_in_sch_imp  = sch_imp_ids - students_sch_ids
both_ids         = students_sch_ids & sch_imp_ids

print("Cobertura por escola:")
print(f"   • Escolas presentes em ambos (com alunos): {len(both_ids)}")
print(f"   • Escolas só em students                : {len(only_in_students)}")
print(f"   • Escolas só em sch (sem alunos)    : {len(only_in_sch_imp)}\n")


AVALIAÇÃO DO MERGE STUDENTS x SCH

Dimensões atuais:
   • students : 4,908 linhas × 12 colunas
   • sch  : 597 linhas × 5 colunas

Não há escolas repetidas.

Resumo do merge:
   • Escolas combinadas:: 4,908 (99.80%)
   • Apenas em sch (escola sem aluno em students): 10 (0.20%)

Cobertura por escola:
   • Escolas presentes em ambos (com alunos): 587
   • Escolas só em students                : 0
   • Escolas só em sch (sem alunos)    : 10



Fazendo o merge definitivo

In [99]:
# many_to_one para garantir estrutura aluno -> escola
students_merged = students_imp.merge(
    sch_imp,
    on="CNTSCHID",
    how="left",
    validate="many_to_one"
)

students_merged.shape


(4908, 16)

In [100]:
students_merged.head()

,CNTSTUID,CNTSCHID,ST004D01T,REPEAT,ESCS,DISCLIMA,BELONG,JOYREAD,SCREADCOMP,SENWT,READ,READ.SE,SC016Q01TA,SC016Q02TA,EDUSHORT,STAFFSHORT
0,7600001,7600614,1,0,-1.647,0.320,16.008,1.230,-4.623,0.308,470.375,22.412,20.000,-0.511,1.251,-6.755
1,7600002,7600190,0,0,2.479,3.700,10.140,-3.060,-4.840,0.342,432.607,23.037,20.000,0.902,-2.900,-4.800
2,7600010,7600048,0,0,1.273,-4.530,15.340,-2.640,-4.460,0.525,428.324,31.055,20.000,1.646,5.700,-0.200
3,7600020,7600444,1,1,-3.170,-1.870,11.850,-1.070,-4.650,0.658,378.231,24.029,18.000,1.000,-2.900,-3.100
4,7600022,7600047,1,1,-3.467,-4.370,3.920,-4.860,-4.520,0.511,419.672,16.979,20.000,1.486,-2.700,-4.800


Vamos adicionar mais um ponto de salvaguarda, criando uma cópia do dataframe resultante do merge final, que chamaremos de `students_final`.

In [115]:
students_final = students_merged.copy()

###  Perfil Agregado das Escolas

Até este ponto, trabalhamos exclusivamente com variáveis no nível do aluno (`ESCS`, `READ`, `gênero`, `repetência`, `clima disciplinar` etc.). Embora essas informações sejam fundamentais para caracterizar trajetórias individuais, elas não capturam diretamente o contexto institucional no qual os estudantes estão inseridos. Para operacionalizar o conceito de "efeito escola" em termos observáveis, precisamos agregar características dos alunos por escola, traduzindo a heterogeneidade institucional em métricas concretas: desempenho médio, perfil socioeconômico médio, clima disciplinar médio, entre outras.

Essa etapa cumpre três funções metodológicas centrais:

- Construção de variáveis de nível 2 (escola): Ao calcular médias ponderadas de `READ`, `ESCS`, `DISCLIMA` e `BELONG` para cada `CNTSCHID`, geramos covariáveis agregadas que podem ser incorporadas aos modelos multiníveis. Isso permite separar o efeito das características individuais (nível 1) do efeito das condições da escola (nível 2).

- Respeito ao desenho amostral do PISA: Utilizamos `SENWT` (peso amostral do estudante) como fator de expansão. Como escolas e alunos são amostrados com probabilidades distintas, aplicar ponderação nas agregações garante que as médias escolares reflitam a população-alvo (estudantes brasileiros de 15 anos matriculados em escolas regulares), e não apenas a amostra observada. Uma média simples trataria igualmente um aluno que representa 10 estudantes da população e outro que representa 200, distorcendo as estimativas.

- Fundamentação para hipóteses substantivas: Perguntas como "escolas com maior `ESCS` médio apresentam melhor desempenho, mesmo após controlar o `ESCS` individual?" ou "o clima disciplinar médio da escola modera o gradiente socioeconômico?" só podem ser respondidas se tivermos acesso a essas variáveis agregadas. Sem elas, qualquer inferência sobre "efeito escola" tenderia a confundir diferenças de composição (quem estuda onde) com diferenças reais de oportunidade educacional.

Para cada escola (CNTSCHID), geramos cinco métricas ponderadas por `SENWT`:

| Métrica         | Descrição                     | Interpretação                                             |
| --------------- | ----------------------------- | --------------------------------------------------------- |
| n_students      | Contagem de alunos na amostra | Tamanho da escola na base (não ponderado)                 |
| read_mean_w     | Proficiência média em leitura | Desempenho médio da escola (escala PISA: ~500 ± 100)      |
| escs_mean_w     | Índice socioeconômico médio   | Contexto socioeconômico do corpo discente (centrado em 0) |
| disclima_mean_w | Clima disciplinar médio       | Qualidade do ambiente escolar percebida pelos alunos      |
| belong_mean_w   | Pertencimento médio           | Senso de comunidade e acolhimento (escala 0–10)           |


Exemplo da lógica da Ponderação:

Situação hipotética: uma escola tem 3 alunos na amostra:

| Aluno | READ | SENWT |
| ----- | ---- | ----- |
| A     | 420  | 100   |
| B     | 450  | 50    |
| C     | 500  | 25    |

#### Média simples (**incorreta**)

$$
\bar{x}_{\text{simples}} = \frac{420 + 450 + 500}{3} = 456{,}67
$$

#### Média ponderada (**correta**)

$$
\bar{x}_{\text{pond}} =
\frac{420 \cdot 100 + 450 \cdot 50 + 500 \cdot 25}{100 + 50 + 25}
= \frac{42000 + 22500 + 12500}{175}
= \frac{77000}{175}
= 440
$$


#### Interpretação

O aluno A representa 4 vezes mais estudantes na população do que o aluno C.
Se ignorarmos os pesos amostrais (SENWT), superestimamos o desempenho médio da escola, pois tratamos como equivalentes observações que têm importâncias muito diferentes no plano amostral.

Para implementar isso, utilizamos a função `groupby("CNTSCHID")` para organizar os dados em grupos de alunos por escola. Em cada grupo, aplicamos a função `estat.wavg(valor, w=SENWT)` para calcular médias ponderadas com os pesos amostrais dos estudantes. Isso garante que cada média reflita não apenas os alunos da amostra, mas a população de estudantes que eles representam. O resultado é um DataFrame de perfil escolar com uma linha por escola, contendo indicadores como tamanho da amostra (`n_students`), proficiência média ponderada em leitura (`read_mean_w`), nível socioeconômico médio (`escs_mean_w`), clima disciplinar (`disclima_mean_w`) e pertencimento (`belong_mean_w`). Esse perfil será, então, mesclado ao dataset de alunos, para ser utilizado como um conjunto de variáveis de contexto em nível da escola.

Depois de refletir sobre a ponderação, concluímos que mesmo os índices derivados, como o `ESCS`, calculados originalmente em nível do estudante, devem ser ponderados ao agregá-los para o nível da escola. Para obter o "ESCS médio da escola", utilizamos a média ponderada pelos pesos dos alunos daquela escola, por exemplo:

$$
ESCS_{\text{escola}} =
\frac{\sum_{i \in \text{escola}} SENWT_i \cdot ESCS_i}
     {\sum_{i \in \text{escola}} SENWT_i}
$$

O mesmo raciocínio se aplica a escalas como clima disciplinar e pertencimento: ponderar por `SENWT` evita que um subconjunto atípico de respondentes distorça o perfil da escola, assegurando que os indicadores contextuais usados nas análises reflitam adequadamente a população-alvo do PISA.



In [116]:
school_profile = (
    students_final
    .groupby("CNTSCHID", as_index=False)
    .apply(
        lambda df: pd.Series({
            "n_students": len(df),
            "read_mean_w": estat.wavg(df["READ"], df["SENWT"]),
            "escs_mean_w": estat.wavg(df["ESCS"], df["SENWT"]),
            "disclima_mean_w": estat.wavg(df["DISCLIMA"], df["SENWT"]),
            "belong_mean_w": estat.wavg(df["BELONG"], df["SENWT"]),
        }),
        include_groups=False 
    )
)

school_profile.head()


,CNTSCHID,n_students,read_mean_w,escs_mean_w,disclima_mean_w,belong_mean_w
0,7600001,8.000,329.420,-0.942,0.354,8.127
1,7600002,9.000,427.369,-1.052,-0.956,6.130
2,7600003,1.000,319.741,0.716,2.090,11.684
3,7600005,12.000,548.127,2.991,0.140,10.795
4,7600006,8.000,400.459,-0.808,-0.590,10.425



Vamos entender o significado de cada métrica gerada no perfil agregado das escolas:

* **CNTSCHID**: É o identificador da escola no PISA. Serve apenas para saber “de qual escola é esta linha”. Não é métrica substantiva.

* **n_students**: É a quantidade de alunos da amostra naquela escola (contagem simples de linhas).
  **Interpretação:** tamanho da amostra escolar no banco, não é número real de alunos da escola na população; é “quantos casos temos para estimar”.

* **read_mean_w**: É a média ponderada da proficiência em leitura dos alunos da escola. Usa os pesos amostrais (SENWT), então respeita o plano amostral do PISA.
  **Escala:** PISA (média ≈ 500, desvio ≈ 100).
  Ex.: 329 indica desempenho bem abaixo da média internacional.

* **escs_mean_w**: É a média ponderada do índice socioeconômico (ESCS) dos alunos da escola.
  **Escala:** centrada em 0 (0 ≈ média dos países da OCDE; valores negativos = contexto menos favorecido; positivos = contexto mais favorecido).
  Ex.: -0.942 sugere escola com alunos em contexto socioeconômico mais baixo.

* **disclima_mean_w**: É a média ponderada do índice de clima disciplinar percebido pelos alunos.
  Geralmente é um índice padronizado (em torno de 0), onde valores maiores indicam clima mais organizado e menos problemas disciplinares; negativos indicam pior clima que a referência.
  Ex.: -0.956 sugere percepção de problemas de disciplina acima do esperado.

* **belong_mean_w**: É a média ponderada do índice de pertencimento (sense of belonging).
  **Escala típica:** algo como 0–10 (dependendo da construção, mas aqui já está em métrica interpretável pela própria documentação).
  Valores maiores indicam maior sensação de acolhimento, integração e identificação com a escola.
  Ex.: 6.130 vs 10.795 → escolas com níveis bem distintos de vínculo percebido pelos alunos.


### Anexar perfil de escola ao nível aluno

Agora que temos o perfil agregado das escolas, podemos incorporá-lo ao dataset de alunos via `CNTSCHID`. Isso nos permitirá incluir variáveis de contexto escolar em nossos modelos multiníveis, enriquecendo a análise do efeito escola sobre o desempenho em leitura.

In [ ]:
students_final = students_final.merge(
    school_profile,
    on="CNTSCHID",
    how="left",
    validate="many_to_one"
)

In [119]:
students_final[[
    "CNTSTUID", "CNTSCHID", "READ", "ESCS",
    "read_mean_w", "escs_mean_w", "disclima_mean_w"
]].head()

,CNTSTUID,CNTSCHID,READ,ESCS,read_mean_w,escs_mean_w,disclima_mean_w
0,7600001,7600614,470.375,-1.647,455.001,0.052,1.980
1,7600002,7600190,432.607,2.479,483.699,0.895,0.825
2,7600010,7600048,428.324,1.273,428.311,-1.893,2.093
3,7600020,7600444,378.231,-3.170,399.373,-0.091,-0.853
4,7600022,7600047,419.672,-3.467,342.109,-2.506,0.132


#### Agora a brincadeira começa: das médias ao efeito escola

Ao mesclar `school_profile` com `students_final`, cada linha passou a combinar informações individuais (`READ`, `ESCS`) com informações contextuais da escola (`read_mean_w`, `escs_mean_w`, `disclima_mean_w`) e essa estrutura nos permite distinguir duas dimensões fundamentais:

1. **Quem estuda na escola** – a composição dos alunos;
2. **Que escola é essa** – o contexto e os possíveis efeitos institucionais.

E com isso, podemos investigar o tal **efeito escola**: escolas com `escs_mean_w` baixo mas `read_mean_w` alto, por exemplo, podem indicar um **efeito escola positivo**, por exemplo? Vejamos:

#### 1. Composição: “Quem estuda aqui?”

Considere duas escolas:

* **Escola A**: maioria dos alunos com ESCS baixo.
* **Escola B**: maioria dos alunos com ESCS alto.

Se olharmos apenas para `read_mean_w` (proficiência média em leitura), não sabemos se a diferença entre A e B vem de:

* características socioeconômicas dos alunos (**composição**),
* práticas, recursos e clima da escola (**efeito escola**),
* ou uma mistura das duas coisas.

Para começar a separar essas dimensões, precisamos ter ao mesmo tempo o **ESCS individual do aluno** ($ESCS_{ij}$) e o **ESCS médio ponderado da escola** ($escs\_mean_{w,j}$), que resume o contexto socioeconômico dos alunos daquela escola.

#### 2. Contexto e efeito escola: "O que a escola faz/é?"

A análise ganha potência quando usamos variáveis em dois níveis no mesmo modelo: nível do aluno ($ESCS_{ij}$) e nível da escola ($escs\_mean_{w,j}$).

Um modelo <span title="Vamos olhar para um modelo multinível simplificado. Aqui estamos modelando a nota de leitura de um aluno i que estuda na escola j. Temos um intercepto, mais o efeito do ESCS individual do aluno, mais o efeito do ESCS médio da escola, mais dois termos de erro: um no nível da escola e outro no nível do aluno. O que esses coeficientes significam? Beta 1 captura o efeito de diferenças de ESCS entre alunos da mesma escola — é o efeito individual, tipo: se dois alunos estudam na mesma escola, mas um tem ESCS mais alto, quanto isso impacta na nota? Já beta 2 é mais interessante: ele captura o efeito de estudar em uma escola com ESCS médio mais alto. Ou seja, mantendo fixo o ESCS individual do aluno, será que estudar numa escola com colegas de contexto socioeconômico mais alto faz diferença? Isso é o que chamamos de efeito contextual ou efeito de composição. E por fim, temos o u_j, o efeito aleatório da escola. Depois de controlar pelo ESCS individual e pelo ESCS médio da escola, ainda sobra algo? Esse resíduo é frequentemente interpretado como parte do efeito escola propriamente dito — o que a escola agrega além da composição dos seus alunos."> multinível</span> simplificado pode ser escrito como:

$$
READ_{ij} =
\beta_0
+ \beta_1 \cdot ESCS_{ij}
+ \beta_2 \cdot escs\_mean_{w,j}
+ u_j
+ e_{ij}
$$

onde:

- $ESCS_{ij}$: capital socioeconômico do aluno $i$ na escola $j$.
- $escs\_mean_{w,j}$ (**coluna `escs_mean_w`**), ou $\overline{ESCS}^{(w)}_{j}$: ESCS médio ponderado dos alunos da escola $j$.
- $u_{j}$: efeito aleatório (desvio) da escola $j$ após controlar pelos demais termos.
- $e_{ij}$: erro individual do aluno $i$ na escola $j$.

Interpretações típicas:

- $\beta_{1}$: efeito de diferenças de $ESCS$ **entre alunos da mesma escola** (efeito individual).
- $\beta_{2}$: efeito de estudar em uma escola com $escs\_mean_{w,j}$ mais alto, mantendo fixo o $ESCS_{ij}$ (efeito contextual/composição).
- $u_{j}$: componente residual em nível da escola, frequentemente interpretado como parte do **efeito escola** (o que a escola agrega além da composição dos alunos).


#### 3. Escolas com `escs_mean_w` baixo e `read_mean_w` alto

Com os agregados disponíveis, podemos investigar casos em que:

* a escola tem `escs_mean_w` baixo (contexto socioeconômico desfavorável),
* mas apresenta `read_mean_w` acima do valor esperado para esse contexto.

Um procedimento simples:

1. Calcular `escs_mean_w` (contexto socioeconômico médio ponderado da escola).
2. Calcular `read_mean_w` (proficiência média ponderada em leitura).
3. Ajustar um modelo que prediga `read_mean_w` a partir de `escs_mean_w`.
4. Observar os **resíduos** desse modelo.

Escolas com resíduos positivos altos (desempenho muito acima do previsto pelo contexto) são candidatas a apresentar um **efeito escola positivo** — sugerem que oferecem condições, práticas ou ambientes que promovem aprendizagem além do que seria esperado apenas pela composição socioeconômica dos alunos.

#### 4. Ligando com o DataFrame na prática

Ao ter, em cada linha do `students_final`:

* variáveis individuais (`READ`, `ESCS`);
* variáveis contextuais da escola (`read_mean_w`, `escs_mean_w`, `disclima_mean_w`),

você consegue:

* separar efeitos **individuais** (quem é o aluno) de efeitos **contextuais** (que escola ele frequenta);
* identificar escolas que “fogem da curva”, positiva ou negativamente;
* especificar modelos multiníveis coerentes com o desenho amostral e com a distinção entre composição e contexto.

Sem esses agregados, qualquer conclusão sobre “efeito escola” arrisca confundir **quem a escola atende** com **o que a escola efetivamente faz**.

## Estatísticas Descritivas (ponderadas)

Utilizamos `SENWT` para aproximar estimativas populacionais do PISA.


In [ ]:
def weighted_summary(df, column, weight):
    clean = df.dropna(subset=[column, weight])
    if clean.empty:
        return {"weighted_mean": np.nan, "weighted_sd": np.nan, "min": np.nan, "max": np.nan, "valid_n": 0}
    w = clean[weight]
    values = clean[column]
    mean = weighted_average(values, w)
    var = weighted_average((values - mean) ** 2, w)
    return {
        "weighted_mean": mean,
        "weighted_sd": np.sqrt(var),
        "min": values.min(),
        "max": values.max(),
        "valid_n": len(values)
    }

summary_metrics = {
    metric: weighted_summary(students, metric, "SENWT")
    for metric in ["READ", "escs_std", "disclima_std"]
}

pd.DataFrame(summary_metrics).T


In [ ]:
# Distribuição das notas de leitura e do índice socioeconômico
sns.histplot(students.dropna(subset=["READ"]), x="READ", bins=30, color="#377eb8")
plt.title("Distribuição de leitura (READ)")
plt.show()

sns.histplot(students.dropna(subset=["escs_std"]), x="escs_std", bins=30, color="#4daf4a")
plt.title("Distribuição do índice socioeconômico (ESCS)")
plt.show()


In [ ]:
# Gradiente socioeconômico com clima disciplinar como cor
sample = students.dropna(subset=["READ", "escs_std", "disclima_std"])
if len(sample) > 3000:
    sample = sample.sample(n=3000, random_state=42)

plt.figure(figsize=(7, 6))
sns.scatterplot(
    data=sample,
    x="escs_std",
    y="READ",
    hue="disclima_std",
    palette="viridis",
    alpha=0.7
)
plt.axhline(sample["READ"].mean(), color="gray", linestyle="--", linewidth=1)
plt.axvline(sample["escs_std"].mean(), color="gray", linestyle="--", linewidth=1)
plt.title("Leitura vs. ESCS (coloração = clima disciplinar)")
plt.show()


## Modelos de Gradiente Socioeconômico


In [ ]:
model_data = students.dropna(
    subset=[
        "READ", "escs_std", "disclima_std", "edushort_std", "staffshort_std",
        "gender_male", "repeat_flag", "SENWT"
    ]
)

model1 = smf.wls(
    formula="READ ~ escs_std",
    data=model_data,
    weights=model_data["SENWT"]
).fit()

model2 = smf.wls(
    formula=(
        "READ ~ escs_std + disclima_std + edushort_std + staffshort_std "
        "+ gender_male + repeat_flag + escs_std:edushort_std"
    ),
    data=model_data,
    weights=model_data["SENWT"]
).fit()

print(model1.summary())
print(model2.summary())


## Modelo Multinível e ICC

Ajustamos um modelo com intercepto aleatório por escola para estimar o coeficiente de correlação intraclasse (ICC), seguido por um modelo com inclinação aleatória de ESCS e moderadores de nível escolar.


In [ ]:
mixed_data = model_data.dropna(subset=["edushort_std", "staffshort_std"]).copy()
mixed_data["escs_centered"] = mixed_data["escs_std"] - mixed_data["escs_std"].mean()

null_model = smf.mixedlm(
    formula="READ ~ 1",
    data=mixed_data,
    groups=mixed_data["CNTSCHID"]
).fit()

var_between = null_model.cov_re.iloc[0, 0]
var_within = null_model.scale
icc = var_between / (var_between + var_within)
print(f"ICC (modelo nulo): {icc:.3f}")

mixed_model = smf.mixedlm(
    formula="READ ~ escs_centered + disclima_std + edushort_std + staffshort_std",
    data=mixed_data,
    groups=mixed_data["CNTSCHID"],
    re_formula="~escs_centered"
).fit(method="lbfgs")

print(mixed_model.summary())


## Observações Finais

- Os identificadores do arquivo `FLT_BRA.xlsx` estavam deslocados em +50.000; o ajuste foi necessário para casar com `STU_BRA.xlsx`.
- Índices armazenados como inteiros (ex.: `ESCS`, `DISCLIMA`, `EDUSHORT`) foram reescalados para as métricas usuais antes das análises.
- As estimativas ponderadas indicam média de leitura em torno de 422 pontos e gradiente socioeconômico positivo (~14,5 pontos por desvio de `ESCS`).
- O ICC do modelo nulo (~0.44) aponta forte heterogeneidade entre escolas; a inclusão de clima disciplinar e escassez reduz a variância entre grupos.


## Referências

- ORGANISATION FOR ECONOMIC CO-OPERATION AND DEVELOPMENT (OECD). *PISA 2018 Technical Report*. Paris: OECD Publishing, 2020. Disponível em: `<https://www.oecd.org/pisa/data/pisa2018technicalreport/>`. Acesso em: 6 nov. 2025.

- ORGANISATION FOR ECONOMIC CO-OPERATION AND DEVELOPMENT (OECD). *PISA 2018 Database*. Paris: OECD, 2019. Disponível em: `<https://www.oecd.org/pisa/data/2018database/>`. Acesso em: 6 nov. 2025.
